In [173]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import boto3
import sagemaker
from sagemaker import get_execution_role

import pandas as pd
import numpy as np

from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker

from sagemaker.feature_store.feature_group import FeatureGroup

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
import os
import io

import psycopg2
import gc

import pandas as pd
import numpy as np

from datetime import datetime

import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt


from lifelines import KaplanMeierFitter
from lifelines import CoxPHFitter, WeibullAFTFitter
from sklearn.pipeline import Pipeline
from lifelines.utils.sklearn_adapter import sklearn_adapter


from sklearn.preprocessing import StandardScaler, Normalizer, StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sagemaker.feature_store.feature_group import FeatureGroup


from sklearn.feature_extraction.text import _VectorizerMixin
from sklearn.feature_selection._base import SelectorMixin
from sklearn import set_config

# import time
# set_config(display='diagram') 


import requests
from requests.auth import HTTPBasicAuth


from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder, StandardScaler, Normalizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, f1_score, precision_recall_fscore_support, roc_curve, precision_recall_curve


%matplotlib inline

In [174]:
df = pd.read_csv('ma_ltv_merged_syn.csv')

In [175]:
df = df[df['application_id'].notna()]
df.reset_index(drop=True)
df = df[df['state'].notna()]
df.reset_index(drop=True)

,application_id,owner_email,application_name,policy_id,owner_id,owner_phone,parent_application_id,sk_referral_flag,bk_product_type,carrier,lead_id,first_name,last_name,submitted_weekday,submitted_day,submitted_month,submitted_year,area_code,age,age_range,jrn_boberdoo_amount,jrn_boberdoo_source,jrn_boberdoo_lead_type,jrn_id,jrn_date,jrn_tracking_file_path,jrn_event_date,jrn_firstparty,jrn_tcpa_universal_id,jrn_is_first_party,jrn_error,jrn_url,jrn_year,jrn_month,jrn_day,jrn_hour,jrn_request_f_name,jrn_request_l_name,jrn_request_email,jrn_request_phone1,jrn_request_address1,jrn_request_tcpa_universal_id,jrn_request_provider,jrn_request_age,jrn_request_dob,jrn_request_smoker,jrn_response_audit_authentic,jrn_response_audit_consumer_five_minutes,jrn_response_audit_consumer_hour,jrn_response_audit_consumer_twelve_hours,jrn_response_audit_consumer_twelve_consumer_day,jrn_response_audit_consumer_week,jrn_response_audit_data_integrity,jrn_response_audit_fields_email,jrn_response_audit_fields_f_name,jrn_response_audit_fields_l_name,jrn_response_audit_fields_phone1,jrn_response_audit_fields_address1,jrn_response_audit_device_five_minutes,jrn_response_audit_device_hour,jrn_response_audit_device_twelve_hours,jrn_response_audit_device_day,jrn_response_audit_device_week,jrn_response_audit_consumer_dupe_check,jrn_response_audit_entity_value,jrn_response_audit_ip_five_minutes,jrn_response_audit_ip_hour,jrn_response_audit_ip_twelve_hours,jrn_response_audit_ip_day,jrn_response_audit_ip_week,jrn_response_audit_lead_age,jrn_response_audit_age,jrn_response_audit_lead_duration,jrn_response_audit_duration,jrn_response_audit_lead_dupe_check,jrn_response_audit_lead_dupe,jrn_response_audit_lead_five_minutes,jrn_response_audit_lead_hour,jrn_response_audit_lead_twelve_hours,jrn_response_audit_lead_day,jrn_response_audit_lead_week,jrn_response_audit_market_leadid_tcpa_disclosure,jrn_response_audit_market_leadid_tcpa_prominence,jrn_response_audit_market_leadid_tcpa_contrast,jrn_response_audit_market_leadid_tcpa_visibility,jrn_response_audit_market_leadid_tcpa_stored,jrn_response_audit_market_leadid_tcpa_capture,jrn_response_audit_market_leadid_tcpa_result,jrn_response_audit_market_leadid_result,jrn_response_audit_market_result,jrn_response_audit_url_value,jrn_response_audit_result,jrn_response_audit_token,zcta_latitude,zcta_longitude,zcta_cdc_all_teeth_lost,zcta_cdc_annual_checkup,zcta_cdc_arthritis,zcta_cdc_binge_drinking,zcta_cdc_cancer_except_skin,zcta_cdc_cervical_cancer_screening,zcta_cdc_cholesterol_screening,zcta_cdc_chronic_kidney_disease,zcta_cdc_colorectal_cancer_screening,zcta_cdc_copd,zcta_cdc_core_preventive_services_for_older_men,zcta_cdc_core_preventive_services_for_older_women,zcta_cdc_coronary_heart_disease,zcta_cdc_current_asthma,zcta_cdc_current_smoking,zcta_cdc_dental_visit,zcta_cdc_depression,zcta_cdc_diabetes,zcta_cdc_general_health,zcta_cdc_health_insurance,zcta_cdc_high_blood_pressure,zcta_cdc_high_cholesterol,zcta_cdc_mammography,zcta_cdc_mental_health,zcta_cdc_obesity,zcta_cdc_physical_health,zcta_cdc_physical_inactivity,zcta_cdc_sleep_lt_7_hours,zcta_cdc_stroke,zcta_cdc_taking_bp_medication,zcta_cms_mapd_aetna_mrkt_share,zcta_cms_mapd_bcbs_mrkt_share,zcta_cms_mapd_cigna_mrkt_share,zcta_cms_mapd_humana_mrkt_share,zcta_cms_mapd_kaiser_mrkt_share,zcta_cms_mapd_lis_mrkt_share,zcta_cms_mapd_mrkt_leader,zcta_cms_mapd_other_mrkt_share,zcta_cms_mapd_penetration_2021,zcta_cms_mapd_penetration_2022,zcta_cms_mapd_penetration_pct_change,zcta_cms_mapd_priority_mrkt_share,zcta_cms_mapd_uhc_mrkt_share,zcta_cms_mapd_wellcare_mrkt_share,zcta_nyt_pct_dem,zcta_nyt_pct_gop,zcta_usc_housing_units,zcta_usc_land_area_m2,zcta_usc_median_home_value,zcta_usc_median_household_income,zcta_usc_occupied_housing_units,zcta_usc_pct_20_24,zcta_usc_pct_25_34,zcta_usc_pct_35_44,zcta_usc_pct_45_54,zcta_usc_pct_55_59,zcta_usc_pct_60_64,zcta_usc_pct_65_74,zcta_usc_pct_65_over,zcta_usc_pct_american_indian_alaska_native,zcta_usc_pct_asian,zcta_usc_pct_black,zcta_usc_pct_

In [203]:
catboost_df = df[['submitted_month', 'submitted_year', 'submitted_day', 'age', 'state', 'LTV']]

In [204]:
# catboost_df['gender'].fillna('M', inplace=True)

In [205]:
# catboost_df.gender.value_counts()

In [206]:
catboost_df['submitted_month'] = catboost_df['submitted_month'].astype(int)
catboost_df['submitted_year'] = catboost_df['submitted_year'].astype(int)
catboost_df['submitted_day'] = catboost_df['submitted_day'].astype(int)

/var/folders/5q/5s79sfds3w7_qbfv5xrwbj6r0000gn/T/ipykernel_2674/2920839890.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  catboost_df['submitted_month'] = catboost_df['submitted_month'].astype(int)
/var/folders/5q/5s79sfds3w7_qbfv5xrwbj6r0000gn/T/ipykernel_2674/2920839890.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  catboost_df['submitted_year'] = catboost_df['submitted_year'].astype(int)
/var/folders/5q/5s79sfds3w7_qbfv5xrwbj6r0000gn/T/ipykernel_2674/2920839890.py:3: SettingWithCopyWarning: 
A 

In [207]:
# X_train.gender.value_counts()

In [208]:
# df['zcta_usc_median_home_value'].fillna(0, inplace=True)
# df['age_range'].fillna('More than 85', inplace=True)
# df['submitted_year'].fillna(2022.0, inplace=True)

In [209]:
from sklearn.model_selection import train_test_split


y = catboost_df.pop('LTV')
X = catboost_df


In [210]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
print("train data size:",X_train.shape)
print("test data size:",X_test.shape)

train data size: (14291, 5)
test data size: (7040, 5)


In [211]:
#List of categorical columns
categoricalcolumns = X.select_dtypes(include=["object"]).columns.tolist()
print("Names of categorical columns : ", categoricalcolumns)
#Get location of categorical columns
cat_features = [X.columns.get_loc(col) for col in categoricalcolumns]
print("Location of categorical columns : ",cat_features)

Names of categorical columns :  ['state']
Location of categorical columns :  [4]


In [212]:
# importing Pool
from catboost import Pool
#Creating pool object for train dataset. we give information of categorical fetures to parameter cat_fetaures
train_data = Pool(data=X_train,
                  label=y_train,
                  cat_features=cat_features
                 )
#Creating pool object for test dataset
test_data = Pool(data=X_test,
                  label=y_test,
                  cat_features=cat_features
                 )

In [213]:
from catboost import CatBoostRegressor

In [214]:
#build model
cat_model = CatBoostRegressor(loss_function='RMSEWithUncertainty',iterations=1000, learning_rate=0.2 ,cat_features=cat_features, verbose=False)
# Fit model 
cat_model.fit( train_data,
               eval_set=test_data,
               plot=True
              )


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [217]:
y_predict= cat_model.predict(X_test)
y_predict

array([[   812.33772849,  56928.30864862],
       [   460.30854457,  33193.41577728],
       [   454.56475645,  28664.14029109],
       ...,
       [   712.34332618, 154872.50396919],
       [   228.54225247,  91138.92857693],
       [   698.60480001, 107267.85564724]])

In [222]:
import math
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import r2_score, mean_absolute_error, median_absolute_error

mean_preds = y_predict[:,0] # mean values predicted 
knowledge = y_predict[:,1] # knowledge uncertainty predicted 
#RMSE
print("predicted",mean_preds)
print("knowledge",knowledge)

Rmse_test = math.sqrt(mean_squared_error(y_test,y_predict[:,0]))
#R2 Score
r2_test = r2_score(y_test,y_predict[:,0])
# Adjusted R2 Score
n= X_train.shape[0] # total no of datapoints
p= X_train.shape[1] # total no of independent features
adj_r2_test = 1-(1-r2_test)*(n-1)/(n-p-1)
#print results
print("Evaluation on test data")
print("RMSE: {:.2f}".format(Rmse_test))
print("R2: {:.2f}".format(r2_test))
print("Adjusted R2: {:.2f}".format(adj_r2_test))


predicted [812.33772849 460.30854457 454.56475645 ... 712.34332618 228.54225247
 698.60480001]
knowledge [ 56928.30864862  33193.41577728  28664.14029109 ... 154872.50396919
  91138.92857693 107267.85564724]
Evaluation on test data
RMSE: 249.91
R2: 0.42
Adjusted R2: 0.42
